In [25]:
print("--------------  Lab0 - Task 0.1 ------------------")

--------------  Lab0 - Task 0.1 ------------------


In [2]:
import sys
print(sys.version)

3.10.16 (main, Dec 11 2024, 10:22:29) [Clang 14.0.6 ]


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

classes = trainset.classes


100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [19]:
""" Simple CNN model with LeakyReLU/Tanh"""

import torch.nn as nn
import torch.nn.functional as F

class CNNLeakyReLU(nn.Module):
    def __init__(self):
        super(CNNLeakyReLU, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)
        self.leakyrelu = nn.LeakyReLU()

    def forward(self, x):
        x = self.pool(self.leakyrelu(self.conv1(x)))
        x = self.pool(self.leakyrelu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.leakyrelu(self.fc1(x))
        x = self.fc2(x)
        return x

class CNNTanh(nn.Module):
    def __init__(self):
        super(CNNTanh, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.tanh(self.conv1(x)))
        x = self.pool(torch.tanh(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        return x




In [21]:
""" Train the Model with a given Optimizer"""

from torch.utils.tensorboard import SummaryWriter



def train_model(model, optimizer, criterion, trainloader, testloader, run_name="experiment", num_epochs=10):
    writer = SummaryWriter(f"runs/{run_name}")

    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        for i, (images, labels) in enumerate(trainloader):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader)
        test_accuracy = evaluate_accuracy(model, testloader)

        print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f} - Test Accuracy: {test_accuracy:.2f}%")

        writer.add_scalar("Loss/train", avg_loss, epoch)
        writer.add_scalar("Accuracy/test", test_accuracy, epoch)

    writer.close()

def evaluate_accuracy(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total



In [22]:
""" Train the model with SGD"""

# With SGD
model = CNNLeakyReLU()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

train_model(model, optimizer, criterion, trainloader, testloader, run_name="sgd_run", num_epochs=10)



Epoch 1 - Loss: 2.3035 - Test Accuracy: 9.86%
Epoch 2 - Loss: 2.3018 - Test Accuracy: 10.35%
Epoch 3 - Loss: 2.3002 - Test Accuracy: 10.87%
Epoch 4 - Loss: 2.2985 - Test Accuracy: 11.71%
Epoch 5 - Loss: 2.2969 - Test Accuracy: 12.65%
Epoch 6 - Loss: 2.2953 - Test Accuracy: 13.55%
Epoch 7 - Loss: 2.2937 - Test Accuracy: 14.13%
Epoch 8 - Loss: 2.2920 - Test Accuracy: 14.57%
Epoch 9 - Loss: 2.2903 - Test Accuracy: 14.99%
Epoch 10 - Loss: 2.2886 - Test Accuracy: 15.66%


In [23]:
""" With Adam"""
# With Adam
model = CNNLeakyReLU()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

train_model(model, optimizer, criterion, trainloader, testloader, run_name="adam_run", num_epochs=10)

Epoch 1 - Loss: 1.5714 - Test Accuracy: 51.70%
Epoch 2 - Loss: 1.2686 - Test Accuracy: 56.11%
Epoch 3 - Loss: 1.1604 - Test Accuracy: 60.18%
Epoch 4 - Loss: 1.0754 - Test Accuracy: 61.60%
Epoch 5 - Loss: 1.0058 - Test Accuracy: 64.13%
Epoch 6 - Loss: 0.9468 - Test Accuracy: 65.03%
Epoch 7 - Loss: 0.8924 - Test Accuracy: 65.50%
Epoch 8 - Loss: 0.8445 - Test Accuracy: 68.03%
Epoch 9 - Loss: 0.7986 - Test Accuracy: 68.82%
Epoch 10 - Loss: 0.7564 - Test Accuracy: 69.67%


In [24]:
"""Swap LeakyReLU with TanH"""

# With SGD
model = CNNTanh()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

train_model(model, optimizer, criterion, trainloader, testloader, run_name="tanh_adam_run", num_epochs=10)

Epoch 1 - Loss: 1.5488 - Test Accuracy: 51.53%
Epoch 2 - Loss: 1.2627 - Test Accuracy: 57.41%
Epoch 3 - Loss: 1.1382 - Test Accuracy: 60.80%
Epoch 4 - Loss: 1.0517 - Test Accuracy: 62.40%
Epoch 5 - Loss: 0.9801 - Test Accuracy: 64.78%
Epoch 6 - Loss: 0.9194 - Test Accuracy: 65.86%
Epoch 7 - Loss: 0.8694 - Test Accuracy: 66.96%
Epoch 8 - Loss: 0.8231 - Test Accuracy: 68.52%
Epoch 9 - Loss: 0.7827 - Test Accuracy: 68.61%
Epoch 10 - Loss: 0.7443 - Test Accuracy: 69.01%


In [ ]:
# Visualize teh Results in TensorBoard

In [ ]:
print("--------------  Lab0 - Task 0.2 ------------------")

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

def get_cifar10_loaders(batch_size=64, resize=False):
    transform_list = []

    if resize:
        transform_list.append(transforms.Resize(224))  # Needed for AlexNet, ResNet, etc.

    transform_list += [
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],   # ImageNet mean
                std=[0.229, 0.224, 0.225]     # ImageNet std
            )
        ]

    transform = transforms.Compose(transform_list)

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

    return trainloader, testloader, trainset.classes



In [ ]:
import torchvision.models as models
import torch.nn as nn

def get_alexnet_model(pretrained=False, feature_extract=False):
    model = models.alexnet(pretrained=pretrained)

    if feature_extract:
        for param in model.parameters():
            param.requires_grad = False

    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, 10)

    return model


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

def train_and_evaluate(model, trainloader, testloader, num_epochs=10, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1} - Loss: {running_loss/len(trainloader):.4f}")

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy


In [ ]:

# Load data
trainloader, testloader, _ = get_cifar10_loaders(resize=False)

# Fine-tune
model_finetune = get_alexnet_model(pretrained=True, feature_extract=False)
train_and_evaluate(model_finetune, trainloader, testloader, num_epochs=3, lr=0.0001)


In [ ]:
# With feature extraction,
trainloader, testloader, _ = get_cifar10_loaders(resize=True)
model_features = get_alexnet_model(pretrained=True, feature_extract=True)
train_and_evaluate(model_features, trainloader, testloader, num_epochs=10, lr=0.0001)

Files already downloaded and verified
Files already downloaded and verified


/Users/nagarajan.ganesan/anaconda3/envs/dl-ex/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nagarajan.ganesan/anaconda3/envs/dl-ex/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[experiment] Epoch 1 - Loss: 1.1262 - Test Accuracy: 71.88%


In [ ]:
print("--------------  Lab0 - Task 0.2.2 ------------------")

In [80]:
"""
MNIST: Grayscale, 28×28 digits (0–9)

SVHN: RGB, 32×32 digits (real-world house numbers)

Training a CNN on MNIST

Using that trained model for SVHN, either by:

Fine-tuning (retraining the whole model), or

Feature extraction (freezing early layers)

"""
# CNN Model
class MNISTCNN(nn.Module):
    def __init__(self):
        super(MNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # 28x28 -> pool -> 14x14 -> pool -> 7x7
        self.fc2 = nn.Linear(128, 10) # For 10 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # [B, 32, 14, 14]
        x = self.pool(torch.relu(self.conv2(x)))  # [B, 64, 7, 7]
        x = x.view(x.size(0), -1)                 # [B, 3136]
        x = torch.relu(self.fc1(x))               # [B, 128]
        x = self.fc2(x)                           # [B, 10]
        return x


import copy

class SVHNModel(nn.Module):
    def __init__(self, pretrained_mnist):
        super(SVHNModel, self).__init__()
        # Copy convolution layers from pretrained MNIST model
        self.conv1 = pretrained_mnist.conv1
        self.conv2 = pretrained_mnist.conv2
        self.pool = pretrained_mnist.pool

        # Replace classifier (new FC layers for SVHN)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x




In [ ]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter


from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def get_dataset_loaders(dataset_name, batch_size=64, resize=True):
    if dataset_name == "mnist":
        transform = transforms.Compose([
            transforms.Resize((28, 28)) if resize else transforms.Lambda(lambda x: x),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))  # Standard MNIST normalization
        ])
        trainset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
        testset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

    elif dataset_name == "svhn":
        transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),  # Convert to 1 channel
            transforms.Resize((28, 28)) if resize else transforms.Lambda(lambda x: x),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        from torchvision.datasets import SVHN
        trainset = SVHN(root="./data", split="train", download=True, transform=transform)
        testset = SVHN(root="./data", split="test", download=True, transform=transform)

    else:
        raise ValueError("Unsupported dataset. Use 'mnist' or 'svhn'.")

    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

    return trainloader, testloader


def train_and_evaluate(model, trainloader, testloader,
                       num_epochs=10,
                       lr=0.001,
                       run_name="experiment",
                       log_dir="runs",
                       optimizer_class=torch.optim.Adam,
                       optimizer_kwargs=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    writer = SummaryWriter(log_dir=f"{log_dir}/{run_name}")
    criterion = nn.CrossEntropyLoss()

    # Set default optimizer kwargs if not provided
    if optimizer_kwargs is None:
        optimizer_kwargs = {}

    optimizer = optimizer_class(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, **optimizer_kwargs)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            #print("Model output shape:", outputs.shape)
            #print("Target shape:", labels.shape)
            loss = criterion(outputs, labels)
            #print("Target shape:", labels.shape)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader)

        # Evaluation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        test_accuracy = 100 * correct / total

        print(f"[{run_name}] Epoch {epoch+1} - Loss: {avg_loss:.4f} - Test Accuracy: {test_accuracy:.2f}%")

        writer.add_scalar("Loss/train", avg_loss, epoch)
        writer.add_scalar("Accuracy/test", test_accuracy, epoch)

    writer.close()


In [86]:
""" Train on MNIST"""
trainloader, testloader = get_dataset_loaders("mnist")
mnist_model = MNISTCNN()
train_and_evaluate(mnist_model,  trainloader, testloader, num_epochs=3,
                   run_name="mnist_adam_run")


[mnist_adam_run] Epoch 1 - Loss: 0.1336 - Test Accuracy: 98.70%


In [85]:
""" Transfer and train SVHN"""
svhn_model = SVHNModel(pretrained_mnist=copy.deepcopy(mnist_model))
svhn_trainloader, svhn_testloader = get_dataset_loaders("svhn")

train_and_evaluate(svhn_model, svhn_trainloader, svhn_testloader, num_epochs=5, lr=0.001, run_name="svhn_transfer")


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
[svhn_transfer] Epoch 1 - Loss: 0.6962 - Test Accuracy: 85.10%
[svhn_transfer] Epoch 2 - Loss: 0.4116 - Test Accuracy: 87.38%
[svhn_transfer] Epoch 3 - Loss: 0.3358 - Test Accuracy: 87.51%
[svhn_transfer] Epoch 4 - Loss: 0.2831 - Test Accuracy: 88.77%
[svhn_transfer] Epoch 5 - Loss: 0.2433 - Test Accuracy: 89.14%


In [90]:
""" Optional: Transfer model"""
import copy

class SVHNTransferCNN(nn.Module):
    def __init__(self, pretrained_mnist):
        super(SVHNTransferCNN, self).__init__()
        # Copy convolutional layers
        self.conv1 = pretrained_mnist.conv1
        self.conv2 = pretrained_mnist.conv2
        self.pool = pretrained_mnist.pool

        # New classification head (trainable)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [91]:
transfer_model = SVHNTransferCNN(pretrained_mnist=copy.deepcopy(mnist_model))

train_and_evaluate(
    transfer_model,
    svhn_trainloader,
    svhn_testloader,
    num_epochs=5,
    lr=0.001,
    run_name="svhn_transfer",
    log_dir="logs/mnist_svhn"
)


[svhn_transfer] Epoch 1 - Loss: 0.6960 - Test Accuracy: 85.45%
[svhn_transfer] Epoch 2 - Loss: 0.4078 - Test Accuracy: 86.78%
[svhn_transfer] Epoch 3 - Loss: 0.3311 - Test Accuracy: 88.65%
[svhn_transfer] Epoch 4 - Loss: 0.2794 - Test Accuracy: 88.83%
[svhn_transfer] Epoch 5 - Loss: 0.2365 - Test Accuracy: 89.61%
